# Choroid Plexus Segmentation Training via Auto3DSeg

In [34]:
import os
import json
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import glob
import random
import platform
from monai.apps.auto3dseg import AutoRunner
from monai.config import print_config
import importlib
from dataclasses import asdict
from loguru import logger
import sys

from reload_recursive import reload_recursive

import mri_data
import monai_training

print_config()

MONAI version: 1.3.2
Numpy version: 1.26.4
Pytorch version: 2.4.0+cu121
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 59a7211070538586369afd4a01eca0a7fe2e742e
MONAI __file__: /home/<username>/.virtualenvs/monai/lib/python3.12/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.11
ITK version: 5.4.0
Nibabel version: 5.2.1
scikit-image version: 0.24.0
scipy version: 1.14.0
Pillow version: 10.4.0
Tensorboard version: 2.17.0
gdown version: 5.2.0
TorchVision version: 0.19.0+cu121
tqdm version: 4.66.4
lmdb version: 1.5.1
psutil version: 6.0.0
pandas version: 2.2.2
einops version: 0.8.0
transformers version: 4.43.3
mlflow version: 2.15.0
pynrrd version: 1.0.0
clearml version: 1.16.2

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [40]:
reload_recursive(monai_training)
reload_recursive(mri_data)

from mri_data.file_manager import scan_3Tpioneer_bids, filter_first_ses
from monai_training.preprocess import DataSetProcesser
from monai_training import training

In [36]:
logger.remove()
logger.add(sys.stderr, level="DEBUG")

2

In [37]:
hostname = platform.node()
if hostname == "rhinocampus":
    drive_root = Path("/media/smbshare")
else:
    drive_root = Path("/mnt/h")

projects_root = Path("/home/srs-9/Projects")
msmri_home = projects_root / "ms_mri"
training_work_dirs = msmri_home / "training_work_dirs"

dataroot = drive_root / "3Tpioneer_bids"
work_dir_name = "choroid_resegment2"
work_dir = training_work_dirs / work_dir_name
modalities = ["flair", "t1"]


## Prep the database

Get the data and labels organized

In [38]:
dataset_proc = DataSetProcesser.new_dataset(dataroot, scan_3Tpioneer_bids, filters=[filter_first_ses])
dataset_proc.prepare_images(modalities)
dataset_proc.prepare_labels(["choroid_t1_flair"], suffix_list=["CH", "ED", "DT"])

dataset = dataset_proc.dataset
dataset.sort()

2024-10-28 22:59:48.129 | INFO     | monai_training.preprocess:prepare_images:107 - Prepare Images
  0%|          | 0/575 [00:00<?, ?it/s]2024-10-28 22:59:48.144 | WARNING  | monai_training.preprocess:prepare_images:138 - Couldn't prepare image for Scan(subid=1229, sesid=20170715)
2024-10-28 22:59:48.194 | WARNING  | monai_training.preprocess:prepare_images:138 - Couldn't prepare image for Scan(subid=1394, sesid=20191012)
2024-10-28 22:59:48.233 | WARNING  | monai_training.preprocess:prepare_images:138 - Couldn't prepare image for Scan(subid=2001, sesid=20161222)
 53%|█████▎    | 303/575 [00:00<00:00, 1006.29it/s]2024-10-28 22:59:48.479 | INFO     | mri_data.utils:merge_images:28 - fslmerge -a /mnt/h/3Tpioneer_bids/sub-ms2120/ses-20170920/flair.t1.nii.gz /mnt/h/3Tpioneer_bids/sub-ms2120/ses-20170920/flair.nii.gz /mnt/h/3Tpioneer_bids/sub-ms2120/ses-20170920/t1.nii.gz


Image Exception : #22 :: Failed to read volume /mnt/h/3Tpioneer_bids/sub-ms2120/ses-20170920/flair.nii.gz
Error : Error: short read, file may be truncated
terminate called after throwing an instance of 'std::runtime_error'
  what():  Failed to read volume /mnt/h/3Tpioneer_bids/sub-ms2120/ses-20170920/flair.nii.gz
Error : Error: short read, file may be truncated


 57%|█████▋    | 329/575 [00:12<00:16, 14.95it/s]  2024-10-28 23:00:00.543 | WARNING  | monai_training.preprocess:prepare_images:138 - Couldn't prepare image for Scan(subid=1379, sesid=20161022)
2024-10-28 23:00:00.549 | INFO     | mri_data.utils:merge_images:28 - fslmerge -a /mnt/h/3Tpioneer_bids/sub-ms1196/ses-20161004/flair.t1.nii.gz /mnt/h/3Tpioneer_bids/sub-ms1196/ses-20161004/flair.nii.gz /mnt/h/3Tpioneer_bids/sub-ms1196/ses-20161004/t1.nii.gz


/usr/local/fsl/share/fsl/bin/fslmerge: line 2: 130928 Aborted                 (core dumped) /usr/local/fsl/bin/fslmerge "$@"
Image Exception : #22 :: Failed to read volume /mnt/h/3Tpioneer_bids/sub-ms1196/ses-20161004/t1.nii.gz
Error : Error: short read, file may be truncated
terminate called after throwing an instance of 'std::runtime_error'
  what():  Failed to read volume /mnt/h/3Tpioneer_bids/sub-ms1196/ses-20161004/t1.nii.gz
Error : Error: short read, file may be truncated


2024-10-28 23:00:12.207 | ERROR    | monai_training.preprocess:prepare_images:147 - Something went wrong merging images for <bound method Scan.info of Scan(subid='1196', sesid='20161004', _dataroot=PosixPath('/mnt/h/3Tpioneer_bids'), _root=PosixPath('/mnt/h/3Tpioneer_bids/sub-ms1196/ses-20161004'), image=None, label=None, cond=None, id=24112560784)>
 84%|████████▎ | 481/575 [00:24<00:06, 15.42it/s]2024-10-28 23:00:12.315 | WARNING  | monai_training.preprocess:prepare_images:138 - Couldn't prepare image for Scan(subid=1245, sesid=20220219)
2024-10-28 23:00:12.334 | WARNING  | monai_training.preprocess:prepare_images:138 - Couldn't prepare image for Scan(subid=2085, sesid=20170130)
2024-10-28 23:00:12.387 | WARNING  | monai_training.preprocess:prepare_images:138 - Couldn't prepare image for Scan(subid=2106, sesid=20170205)


/usr/local/fsl/share/fsl/bin/fslmerge: line 2: 131040 Aborted                 (core dumped) /usr/local/fsl/bin/fslmerge "$@"


100%|██████████| 575/575 [00:24<00:00, 23.68it/s]
2024-10-28 23:00:12.416 | INFO     | monai_training.preprocess:prepare_labels:167 - Prepare Labels
  0%|          | 0/565 [00:00<?, ?it/s]2024-10-28 23:00:12.421 | DEBUG    | mri_data.file_manager:find_label:397 - Testing choroid_t1_flair-ch.nii.gz
2024-10-28 23:00:12.421 | DEBUG    | mri_data.file_manager:find_label:397 - Testing choroid_t1_flair-ed.nii.gz
2024-10-28 23:00:12.422 | DEBUG    | mri_data.file_manager:find_label:397 - Testing choroid_t1_flair-dt.nii.gz
2024-10-28 23:00:12.422 | DEBUG    | mri_data.file_manager:find_label:404 - No label in [] matched search
2024-10-28 23:00:12.424 | WARNING  | monai_training.preprocess:prepare_labels:223 - Couldn't prepare image for Scan(subid=1417, sesid=20170701)
2024-10-28 23:00:12.428 | DEBUG    | mri_data.file_manager:find_label:397 - Testing choroid_t1_flair-ch.nii.gz
2024-10-28 23:00:12.429 | DEBUG    | mri_data.file_manager:find_label:397 - Testing choroid_t1_flair-ed.nii.gz
2024-10

In [39]:
dataset[0]

Scan(subid='1010', sesid='20180208', _dataroot=PosixPath('/mnt/h/3Tpioneer_bids'), _root=PosixPath('/mnt/h/3Tpioneer_bids/sub-ms1010/ses-20180208'), image='flair.t1.nii.gz', label='choroid_t1_flair-CH.nii.gz', cond=None, id=20382010080)

In [41]:
dataset = training.assign_conditions(dataset, 0.2)
dataset[0]

Scan(subid='1010', sesid='20180208', _dataroot=PosixPath('/mnt/h/3Tpioneer_bids'), _root=PosixPath('/mnt/h/3Tpioneer_bids/sub-ms1010/ses-20180208'), image='flair.t1.nii.gz', label='choroid_t1_flair-CH.nii.gz', cond='tr', id=20382010080)

In [42]:
training_data = []
test_data = []

for scan in dataset:
    if scan.cond == 'tr':
        training_data.append({"image": scan.image, "label": scan.label})
    elif scan.cond == 'ts':
        test_data.append(scan.image)

## Review

In [48]:
def display_slices(scan):
    img1 = nib.load(scan.image_path)
    img2 = nib.load(scan.label_path)

    data1 = img1.get_fdata()[:,:,:]
    data2 = img2.get_fdata()

    slice_sums = np.sum(data2, axis=(0, 1))
    
    print(slice_sums)

    max_slice_index = np.argmax(slice_sums)
    print(f"Max slice: {max_slice_index}")

    slice1 = data1[:, :, max_slice_index]
    slice2 = data2[:, :, max_slice_index]    

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(slice1, cmap='gray')
    plt.title(f"Image 1 - Slice {max_slice_index}")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(slice2, cmap='gray')
    plt.title(f"Image 2 - Slice {max_slice_index}")
    plt.axis('off')
    plt.show()


In [46]:
img2 = nib.load(dataset[0].label_path)
data2 = img2.get_fdata()

In [ ]:
display_slices(dataset[20])

In the original code, they include labels in the test data as well. Also they have a function that checks that there is nonzero number of voxels in the label

In [50]:
#? I don't know why they put labels for the test data. the brats segmentation code didn't.
# train_data = [{'image': path + '/flair.nii.gz', 'label': path + '/flair_chp_mask_qced.nii.gz'} for path in train_exams]
# test_data = [{'image': path + '/flair.nii.gz', 'label': path + '/flair_chp_mask_qced.nii.gz'} for path in test_exams]

train_data = []
test_data = []
for scan in dataset:
    if scan.cond == 'tr' and scan.has_label:
        train_data.append({"image": str(scan.image), "label": str(scan.label)})
    elif scan.cond == 'ts' and scan.has_label():
        test_data.append({"image": str(scan.image), "label": str(scan.label)})


print(f"Train num total: {len(train_data)}")
print(f"Test num: {len(test_data)}")

Train num total: 29
Test num: 7


In [51]:
n_folds = 5
datalist = {
    "testing": test_data,
    "training": [{"fold": i % n_folds, "image": c["image"], "label": c["label"]} for i,c in enumerate(train_data)]
}

In [22]:
# sub_datalist = dict({'training':[], 'testing':[]})
# sub_datalist["training"] = datalist["training"][:100]
# sub_datalist["testing"] = datalist["testing"][:29]
# datalist = sub_datalist

In [52]:
datalist

{'testing': [{'image': 'flair.t1.nii.gz',
   'label': 'choroid_t1_flair-ED.nii.gz'},
  {'image': 'flair.t1.nii.gz', 'label': 'choroid_t1_flair-ED.nii.gz'},
  {'image': 'flair.t1.nii.gz', 'label': 'choroid_t1_flair-ED.nii.gz'},
  {'image': 'flair.t1.nii.gz', 'label': 'choroid_t1_flair-ED.nii.gz'},
  {'image': 'flair.t1.nii.gz', 'label': 'choroid_t1_flair-CH.nii.gz'},
  {'image': 'flair.t1.nii.gz', 'label': 'choroid_t1_flair-CH.nii.gz'},
  {'image': 'flair.t1.nii.gz', 'label': 'choroid_t1_flair-ED.nii.gz'}],
 'training': [{'fold': 0,
   'image': 'flair.t1.nii.gz',
   'label': 'choroid_t1_flair-CH.nii.gz'},
  {'fold': 1,
   'image': 'flair.t1.nii.gz',
   'label': 'choroid_t1_flair-CH.nii.gz'},
  {'fold': 2,
   'image': 'flair.t1.nii.gz',
   'label': 'choroid_t1_flair-CH.nii.gz'},
  {'fold': 3,
   'image': 'flair.t1.nii.gz',
   'label': 'choroid_t1_flair-CH.nii.gz'},
  {'fold': 4,
   'image': 'flair.t1.nii.gz',
   'label': 'choroid_t1_flair-ED.nii.gz'},
  {'fold': 0,
   'image': 'flair.t1.

In [53]:
len(datalist['training'])

29

In [54]:
work_dir = "/home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment2"
# work_dir = "/home/hemondlab/Dev/ms_mri/training_work_dirs/cp_work_dir6"
if not os.path.isdir(work_dir):
    os.makedirs(work_dir)

dataroot_dir = "/mnt/h"
if not os.path.isdir(dataroot_dir):
    os.makedirs(dataroot_dir)

datalist_file = os.path.join(work_dir, "datalist.json")
with open(datalist_file, "w") as f:
    json.dump(datalist, f)

In [55]:
runner = AutoRunner(
    work_dir=work_dir,
    algos=["swinunetr"],
    input={
        "modality": "MRI",
        "datalist": datalist_file,
        "dataroot": dataroot_dir,
    },
)

2024-10-28 23:06:16,435 - INFO - AutoRunner using work directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment2
2024-10-28 23:06:16,438 - INFO - Found num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment2/datalist.json.
2024-10-28 23:06:16,439 - INFO - Setting num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment2/datalist.json.
2024-10-28 23:06:16,478 - INFO - Using user defined command running prefix , will override other settings


In [56]:
max_epochs = 100

train_param = {
    "num_epochs_per_validation": 1,
    #"num_images_per_batch": 2,
    "num_epochs": max_epochs,
    "num_warmup_epochs": 1,
}
runner.set_training_params(train_param)

In [24]:
runner.run()

2024-08-29 21:30:57,464 - INFO - Running data analysis...
2024-08-29 21:30:57,465 - INFO - Found 1 GPUs for data analyzing!


100%|██████████| 30/30 [01:06<00:00,  2.23s/it]

2024-08-29 21:32:04,509 - INFO - Data spacing is not completely uniform. MONAI transforms may provide unexpected result
2024-08-29 21:32:04,510 - INFO - Writing data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/cp_work_dir_choroid1/datastats.yaml.
2024-08-29 21:32:04,516 - INFO - Writing by-case data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/cp_work_dir_choroid1/datastats_by_case.yaml, this may take a while.
2024-08-29 21:32:04,648 - INFO - BundleGen from https://github.com/Project-MONAI/research-contributions/releases/download/algo_templates/e4cf5a1.tar.gz



algo_templates.tar.gz: 104kB [00:00, 266kB/s]                              

2024-08-29 21:32:05,052 - INFO - Downloaded: /tmp/tmpxyi0ngdm/algo_templates.tar.gz
2024-08-29 21:32:05,052 - INFO - Expected md5 is None, skip md5 check for file /tmp/tmpxyi0ngdm/algo_templates.tar.gz.
2024-08-29 21:32:05,052 - INFO - Writing into directory: /home/srs-9/Projects/ms_mri/training_work_dirs/cp_work_dir_choroid1.
2024-08-29 21:32:05,108 - INFO - Generated:/home/srs-9/Projects/ms_mri/training_work_dirs/cp_work_dir_choroid1/swinunetr_0
2024-08-29 21:32:05,142 - INFO - Generated:/home/srs-9/Projects/ms_mri/training_work_dirs/cp_work_dir_choroid1/swinunetr_1
2024-08-29 21:32:05,182 - INFO - Generated:/home/srs-9/Projects/ms_mri/training_work_dirs/cp_work_dir_choroid1/swinunetr_2


...

/home/srs-9/.virtualenvs/monai/lib/python3.12/site-packages/ignite/handlers/checkpoint.py:17: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer
monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().


2024-08-30 00:30:22,148 - INFO - Downloaded: /home/srs-9/Projects/ms_mri/training_work_dirs/cp_work_dir_choroid1/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt
2024-08-30 00:30:22,148 - INFO - Expected md5 is None, skip md5 check for file /home/srs-9/Projects/ms_mri/training_work_dirs/cp_work_dir_choroid1/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt.


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
2024/08/30 00:30:22 INFO mlflow.

2024-08-30 01:23:23,826 - INFO - Ensembling using single GPU!
2024-08-30 01:23:23,826 - INFO - The output_dir is not specified. /home/srs-9/Projects/ms_mri/training_work_dirs/cp_work_dir_choroid1/ensemble_output will be used to save ensemble predictions.
2024-08-30 01:23:23,827 - INFO - Directory /home/srs-9/Projects/ms_mri/training_work_dirs/cp_work_dir_choroid1/ensemble_output is created to save ensemble predictions
2024-08-30 01:23:23,863 - INFO - Auto3Dseg picked the following networks to ensemble:
2024-08-30 01:23:23,863 - INFO - swinunetr_0
2024-08-30 01:23:23,864 - INFO - swinunetr_1
2024-08-30 01:23:23,865 - INFO - swinunetr_2
2024-08-30 01:23:23,865 - INFO - swinunetr_3
2024-08-30 01:23:23,866 - INFO - swinunetr_4
2024-08-30 01:23:23,866 - INFO - Auto3Dseg ensemble prediction outputs will be saved in /home/srs-9/Projects/ms_mri/training_work_dirs/cp_work_dir_choroid1/ensemble_output.


Ensembling (rank 0)...:   0%|          | 0/7 [00:00<?, ?it/s]monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `we

In [29]:
scan_path = "/mnt/t/Data/3Tpioneer_bids/sub-ms1001/ses-20170215/proc/lesion_index.t3m20-mni_reg.nii.gz"
img = nib.load(scan_path)
